## Conexao Banco de dados

In [29]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

usuario = 'postgres'
senha = quote_plus('@manaus')
host = 'localhost'
porta = '5432'
nome_do_banco = 'db_mrp'

# Configuração da conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{nome_do_banco}')

### Preparação de Dados para Banco de Dados
Este notebook demonstra o processo de preparação de uma planilha para inserção em um banco de dados. Os passos incluem:

- Carregar o arquivo Excel
- Limpar e ajustar os dados
- Tratar valores nulos
- Exportar os dados para um banco de dados SQL#

## 1. Carregar o arquivo Excel

In [30]:
import pandas as pd

# Carregar a planilha

file_path = r'C:\Users\Paulo\Documents\project01\data\1213_Bom_Master.xlsx'
xls = pd.ExcelFile(file_path)

# Carregar a planilha específica
df_bom = xls.parse('BOM_Master', header=0)

# Exibir as primeiras linhas
df_bom.head()

,Top Item,Top Desc,Level,Parent Item,Parent UIT,Child Item,Child Desc,Child Spec,Child UIT,Child Rev,...,LOCAL,Assy,Supplier,Line,Planner,Purchaser,Supplier.1,Supplier Name,Model MRP,2024-12-13 00:00:00
0,32LR650BPSA.AWZSLJZ,LED LCD TV 32 (HD),1,32LR650BPSA.AWZSLJZ,F,RAB33632804,"Tape,Polyester",POLYESTER 100% FR(FIRE RESISTANCE)/// GRAY 200...,D,0,...,FA,32LR650BPSA.AWZSLJZ,NaN,NaN,KMF01,"SAMPAIO, SORAIA",KR131893,서브원,32LR650BPSA/32LR651CBSA/43TU801C0SA/43UM767H0S...,FA;
1,32LR650BPSA.AWZSLJZ,LED LCD TV 32 (HD),1,32LR650BPSA.AWZSLJZ,F,AAN30117605,Base Assembly,STAND 32LR65 NONE STAND ASSY 2Pole_Rocky BK_Br...,R,1,...,FA,32LR650BPSA.AWZSLJZ,NaN,NaN,WTLD,"ALMEIDA, WITALO",BR010767,TAE YANG DO BRASIL INDUSTRIA E COMERCIO DE INJ...,32LR650BPSA/32LR651CBSA/,FA;
2,32LR650BPSA.AWZSLJZ,LED LCD TV 32 (HD),2,AAN30117605,R,MCQ68894011,Damper,CUTTING TPE FOOT RUBBER TPS semi-transparent 1...,D,0,...,OSP,AAN30117605,BR010767,TYANG,KMF01,"SAMPAIO, SORAIA",KR131893,서브원,32LR650BPSA/32LR651CBSA/70NANO80TSA/70UT8050PS...,OSP;
3,32LR650BPSA.AWZSLJZ,LED LCD TV 32 (HD),3,MAM66016201,P,RAA35297804,"Resin,ABS",Rocky Black G8Q378 G8Q378 29.77 190m -0.75 XG5...,D,0,...,OSP,AAN30117605,BR010767,TYANG,DMF01,"MALCHER, LEANDRO",KR011181,"LG Chem, Ltd.",32LR650BPSA/32LR651CBSA/43UR781C0SA/43UT8000PS...,OSP; Intern;
4,32LR650BPSA.AWZSLJZ,LED LCD TV 32 (HD),2,AAN30117605,R,MEZ64114703,Label,PRINTING OTHERS BRAND Accessory Barcode Label ...,G,0,...,OSP,AAN30117605,BR010767,TYANG,JSL,"LEAL, JULIANE",BR001217,LABEL PRESS IND. E COM. DA AMAZONIA LTDA,32LR650BPSA/32LR651CBSA/,OSP;


In [31]:
df_bom.columns

Index([         'Top Item',          'Top Desc',             'Level',
             'Parent Item',        'Parent UIT',        'Child Item',
              'Child Desc',        'Child Spec',         'Child UIT',
               'Child Rev',       'Supply Type',       'Child C-OSP',
                'Make/Buy',         'Desig No.',           'ECO No.',
                'Quantity', 'Extended Quantity',        'Start Date',
                'End Date',              'Org.',          'Comments',
             'Subs Item 1',       'Subs Item 2',       'Subs Item 3',
             'Subs Item 4',       'Subs Item 5',       'Subs Item 6',
             'Subs Item 7',       'Subs Item 8',       'Subs Item 9',
                     'QPA',        'Class Code',        'Class Name',
                     'UOM',          'SVC Code',      'SVC Location',
                    'Code',             'Code2',             'LOCAL',
                    'Assy',          'Supplier',              'Line',
                 'Pl

In [32]:
# manter as colunas
columns_to_keep = ['Org.', 'Child Item', 'Child Desc', 'Child UIT', 'QPA', 
                   'LOCAL', 'Assy', 'Planner', 'Purchaser', 'Supplier', 
                   'Supplier Name', 'Model MRP']

df_bom = df_bom[columns_to_keep]
df_bom['Infor'] = 'Plano'
df_bom

,Org.,Child Item,Child Desc,Child UIT,QPA,LOCAL,Assy,Planner,Purchaser,Supplier,Supplier Name,Model MRP,Infor
0,NW1,RAB33632804,"Tape,Polyester",D,1.000,FA,32LR650BPSA.AWZSLJZ,KMF01,"SAMPAIO, SORAIA",NaN,서브원,32LR650BPSA/32LR651CBSA/43TU801C0SA/43UM767H0S...,Plano
1,NW1,AAN30117605,Base Assembly,R,1.000,FA,32LR650BPSA.AWZSLJZ,WTLD,"ALMEIDA, WITALO",NaN,TAE YANG DO BRASIL INDUSTRIA E COMERCIO DE INJ...,32LR650BPSA/32LR651CBSA/,Plano
2,NW1,MCQ68894011,Damper,D,4.000,OSP,AAN30117605,KMF01,"SAMPAIO, SORAIA",BR010767,서브원,32LR650BPSA/32LR651CBSA/70NANO80TSA/70UT8050PS...,Plano
3,NW1,RAA35297804,"Resin,ABS",D,0.064,OSP,AAN30117605,DMF01,"MALCHER, LEANDRO",BR010767,"LG Chem, Ltd.",32LR650BPSA/32LR651CBSA/43UR781C0SA/43UT8000PS...,Plano
4,NW1,MEZ64114703,Label,G,1.000,OSP,AAN30117605,JSL,"LEAL, JULIANE",BR010767,LABEL PRESS IND. E COM. DA AMAZONIA LTDA,32LR650BPSA/32LR651CBSA/,Plano
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86129,NWX,MCQ66576505,Damper,D,1.000,OSP,AAX77008901,KITFA,"ALMEIDA, GUILHERME",BR001568,乐金电子(惠州)有限公司,XL9T/,Plano
86130,NWX,MDP64335401,Foot,D,4.000,OSP,AAX77008901,KITFA,"ALMEIDA, GUILHERME",BR001568,乐金电子(惠州)有限公司,XL9T/,Plano
86131,NWX,FAB30093410,"Screw,Customized",D,28.000,OSP,AAX77008901,KITFA,"ALMEIDA, GUILHERME",BR001568,乐金电子(惠州)有限公司,XL9T/,Plano
86132,NWX,1SZZR-0097K,"Screw,Customized",D,10.000,OSP,AAX77008901,KITFA,"ALMEIDA, GUILHERME",BR001568,乐金电子(惠州)有限公司,XG5QBK/XL5S/XL5T/XL7S/XL7T/XL9T/,Plano


In [33]:
import psycopg2
import pandas as pd

# Query para selecionar as colunas 'Date' e 'Quantity' de sua tabela
sql_query = """
SELECT Date, Quantity
FROM table_pph_teste
"""

# Passar a consulta e o parâmetro corretamente para o pandas
df_reference_data = pd.read_sql(sql_query, engine)

df_bom['date'] = df_reference_data['date']
df_bom['quantity'] = df_reference_data['quantity']

df_bom.head(20)


,Org.,Child Item,Child Desc,Child UIT,QPA,LOCAL,Assy,Planner,Purchaser,Supplier,Supplier Name,Model MRP,Infor,date,quantity
0,NW1,RAB33632804,"Tape,Polyester",D,1.000,FA,32LR650BPSA.AWZSLJZ,KMF01,"SAMPAIO, SORAIA",NaN,서브원,32LR650BPSA/32LR651CBSA/43TU801C0SA/43UM767H0S...,Plano,2024-12-01 03:00:00,0.0
1,NW1,AAN30117605,Base Assembly,R,1.000,FA,32LR650BPSA.AWZSLJZ,WTLD,"ALMEIDA, WITALO",NaN,TAE YANG DO BRASIL INDUSTRIA E COMERCIO DE INJ...,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
2,NW1,MCQ68894011,Damper,D,4.000,OSP,AAN30117605,KMF01,"SAMPAIO, SORAIA",BR010767,서브원,32LR650BPSA/32LR651CBSA/70NANO80TSA/70UT8050PS...,Plano,2024-12-01 03:00:00,0.0
3,NW1,RAA35297804,"Resin,ABS",D,0.064,OSP,AAN30117605,DMF01,"MALCHER, LEANDRO",BR010767,"LG Chem, Ltd.",32LR650BPSA/32LR651CBSA/43UR781C0SA/43UT8000PS...,Plano,2024-12-01 03:00:00,0.0
4,NW1,MEZ64114703,Label,G,1.000,OSP,AAN30117605,JSL,"LEAL, JULIANE",BR010767,LABEL PRESS IND. E COM. DA AMAZONIA LTDA,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
5,NW1,MAF62189840,Bag,D,1.000,OSP,AAN30117605,KMF01,"SAMPAIO, SORAIA",BR010767,서브원,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
6,NW1,EAY65912101,Power Supply Assembly,D,1.000,FA,32LR650BPSA.AWZSLJZ,KMD01,"CARDOSO, CLILIAN",NaN,(주)와이투솔루션,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
7,NW1,MEV67788301,Insulator,D,1.000,FA,32LR650BPSA.AWZSLJZ,KMF01,"SAMPAIO, SORAIA",NaN,서브원,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
8,NW1,ACQ30597516,"Cover Assembly,Rear",T,1.000,FA,32LR650BPSA.AWZSLJZ,<NULL>,<NULL>,NaN,0,32LR650BPSA/32LR651CBSA/,Plano,2024-12-01 03:00:00,0.0
9,NW1,RAA36433601,"Resin,HIPS",D,0.820,Intern,ACQ30597516,DMF01,"MALCHER, LEANDRO",,금호석유화학(주)울산합성수지공장,32LR650BPSA/32LR651CBSA/43LR6700PSA/43LR671C0SA/,Plano,2024-12-01 03:00:00,0.0


#### Salvar no banco

In [34]:
# Exportar o DataFrame para o banco de dados PostgreSQL
df_bom.to_sql('table_bom', con=engine, if_exists='replace', index=False)
print('Dados salvos com sucesso no banco de dados PostgreSQL.')

Dados salvos com sucesso no banco de dados PostgreSQL.


In [35]:
df_bom.columns

Index(['Org.', 'Child Item', 'Child Desc', 'Child UIT', 'QPA', 'LOCAL', 'Assy',
       'Planner', 'Purchaser', 'Supplier', 'Supplier Name', 'Model MRP',
       'Infor', 'date', 'quantity'],
      dtype='object')

In [36]:
# Transforma os nomes das colunas para minúsculas
df_bom.columns = df_bom.columns.str.lower()

# Exibe o DataFrame com as colunas em minúsculas
df_bom.columns

Index(['org.', 'child item', 'child desc', 'child uit', 'qpa', 'local', 'assy',
       'planner', 'purchaser', 'supplier', 'supplier name', 'model mrp',
       'infor', 'date', 'quantity'],
      dtype='object')